In [1]:
import train_nn

In [2]:
import random, os
import numpy as np
from config import *
import tensorflow as tf
from tf_graph import FlappyGraph

In [3]:
def add_jumps_to_training(training_images, last_jumps):
    print("Parsing data...")
    iter_counter = 0
    X_data = []
    # (game frames, height, width)
    for i, game in enumerate(training_images):
        X_data.append([])
        for j, image in enumerate(game):
            X_data[iter_counter].append(np.append(image.ravel(), last_jumps[i][j]))
        X_data[iter_counter] = np.array(X_data[iter_counter], dtype=np.float32)
        iter_counter += 1

    return np.array(X_data)

In [4]:
print("Loading data...")
training_images = np.load(os.path.join(DATA_DIR, "images.npy"))
actions = np.load(os.path.join(DATA_DIR, "actions.npy"))
rewards = np.load(os.path.join(DATA_DIR, "adjusted_rewards.npy"))
last_jumps = np.load(os.path.join(DATA_DIR, "last_jumps.npy"))
X_data = add_jumps_to_training(training_images = training_images, last_jumps = last_jumps)

Loading data...
Parsing data...


In [9]:
flappy_graph = FlappyGraph(int((CANVAS_WIDTH * IMG_SCALE_FACTOR) * round(CANVAS_HEIGHT * IMG_SCALE_FACTOR)) + 1, 2)
init = tf.global_variables_initializer()

In [10]:
sess = tf.Session()
sess.run(init)    

In [13]:
for i in range(5):
    for j in range(5):
        _, train_loss = sess.run([flappy_graph.train_step, flappy_graph.loss], 
                    feed_dict={
                        flappy_graph.inputs: X_data[i], 
                        flappy_graph.actions: actions[i], 
                        flappy_graph.rewards: rewards[i], 
                        flappy_graph.lr: 0.0001}
                    )
        print("loss", train_loss)

21231.4
-1.51087e+06
11427.6
21779.8
-764.583


In [14]:
saver = tf.train.Saver()

if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
    saver.save(sess, os.path.join(MODEL_DIR, "trained_flappy"))